In [1]:
import bugzilla
import requests
import time
from datetime import datetime

In [2]:
pagure_owner_alias = requests.get("https://src.fedoraproject.org/extras/pagure_owner_alias.json").json()["rpms"]

In [3]:
orphaned = [p for p in pagure_owner_alias if "orphan" in pagure_owner_alias[p]]
orphaned[:10]

['7kaa',
 '8Kingdoms',
 '915resolution',
 '99soft-oss-parent',
 'AcetoneISO2',
 'Ajaxterm',
 'CableSwig',
 'Canna',
 'CastPodder',
 'CodeAnalyst-gui']

In [4]:
bzapi = bugzilla.Bugzilla("bugzilla.redhat.com")

In [5]:
if not bzapi.logged_in:
    bzapi.interactive_login()

In [6]:
TRACKER = 1750908

query = bzapi.build_query(product="Fedora", include_fields=["flags", "assigned_to", "component", "creation_time"])
query["blocks"] = TRACKER
query["status"] = "NEW"
bugzillas = bzapi.query(query)
len(bugzillas)

287

In [7]:
RELENG = "releng@fedoraproject.org"
ORPHAN = "extras-orphan@fedoraproject.org"

In [8]:
pkgs_to_orphan = []
bugs_to_reassign = []

In [9]:
for bug in bugzillas:
    if bug.component in orphaned:
        if bug.assigned_to != ORPHAN:
            bugs_to_reassign.append(bug.id)
        continue
    needinfos = bug.get_flags("needinfo") or []
    for needinfo in needinfos:
        if needinfo['requestee'] == bug.assigned_to:
            break
    else:
        # no needinfo on assignee
        continue
    bug_state = 0
    last_time = datetime.fromtimestamp(time.mktime(bug.creation_time.timetuple()))
    for comment in bug.getcomments():
        if comment["creator"] != RELENG:
            continue
        if not comment["text"].startswith("Dear Maintainer,"):
            continue
        comment_time = datetime.fromtimestamp(time.mktime(comment["creation_time"].timetuple()))
        delta = comment_time - last_time
        if bug_state == 0 and delta.days >= 7-1:
            last_time = comment_time
            bug_state = 1
        elif bug_state == 1 and delta.days >= 3*7-1:
            last_time = comment_time
            bug_state = 2
            break
    delta = datetime.now() - last_time
    if bug_state == 2 and delta.days >= 4*7-1:
        to_orphan = f"rpms/{bug.component}"
        pkgs_to_orphan.append(to_orphan)
        print(to_orphan)
        bugs_to_reassign.append(bug.id)

rpms/pam_url
rpms/nodejs-flat-cache
rpms/nodejs-url2
rpms/eclipse-webtools
rpms/python-poppler-qt4
rpms/drbd
rpms/oshinko-cli
rpms/python-bashate
rpms/foma
rpms/nodejs-snockets
rpms/eclipse-ptp
rpms/lterm
rpms/nodejs-gnode
rpms/rsyntaxtextarea
rpms/nodejs-q
rpms/nodejs-i
rpms/sscep
rpms/ubertooth
rpms/uwsgi
rpms/fcoe-utils
rpms/repmgr
rpms/rubygem-yell
rpms/svnkit
rpms/mboxgrep
rpms/ocitools
rpms/nodejs-gitconfiglocal
rpms/nodejs-jasmine-growl-reporter
rpms/nodejs-json-parse-helpfulerror
rpms/rigsofrods
rpms/scilab
rpms/nodejs-fs-promise
rpms/nodejs-qunit-extras
rpms/nodejs-underscore-dot-logger
rpms/gcolor2
rpms/apbs
rpms/tftp
rpms/xneur
rpms/ckermit
rpms/nodejs-source-map-support
rpms/nodejs-collections
rpms/uom-se
rpms/nodejs-css-select
rpms/cdpr
rpms/eclipse-m2e-core
rpms/nodejs-is-arrow-function
rpms/docco
rpms/abcm2ps
rpms/graphite2
rpms/nodejs-opal-runtime
rpms/snapraid
rpms/fedora-jam-kde-theme
rpms/jchart2d
rpms/publican-jboss
rpms/hcc
rpms/nodejs-ltx
rpms/nodejs-netmask
rpms/

In [10]:
print("https://bugzilla.redhat.com/buglist.cgi?bug_id=" + ",".join(str(b) for b in bugs_to_reassign))

https://bugzilla.redhat.com/buglist.cgi?bug_id=1799845,1799725,1799801,1799315,1799944,1799290,1799841,1799909,1799362,1799784,1799306,1799618,1799731,1799983,1799774,1799744,1800112,1793493,1794335,1799336,1799974,1800045,1800174,1799639,1799821,1799728,1799749,1799750,1799978,1800077,1799726,1799777,1799799,1799390,1799157,1800195,1800275,1799227,1799792,1799700,1800222,1799706,1799214,1799303,1799747,1799287,1799534,1799137,1799180,1799470,1799769,1800100,1799338,1799537,1799904,1799505,1799756,1799764,1800043,1800253,1799647,1799676,1799164,1799751,1799704,1799400,1800116,1799305,1799708,1799766,1799757,1800223,1800257,1800087,1799291,1799177,1799587,1799785,1799238,1799140,1799831,1799716,1799727,1799580,1799781,1799503,1799695,1799535,1799640,1799848,1785843,1799733,1799881,1799573,1799723,1799654,1799737,1799833,1799698,1799636,1799692,1799721,1799699,1799803,1799667,1799762,1800245,1799988,1799376,1799719,1800286,1799977,1799193,1799500,1799713,1799310,1800260,1800193,1799748,1